In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
import folium

In [ ]:
data = pd.read_csv("squirrel_data.csv")

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
df = data[['X', 'Y',
       'Hectare Squirrel Number', 'Age', 'Primary Fur Color']]

In [ ]:
df['Age'] = df['Age'].replace('?', 'Adult')
df['Age'] = df['Age'].fillna('Adult')

In [ ]:
df = df.rename(columns={'X': 'Longitude'})
df = df.rename(columns={'Y': 'Latitude'})

In [ ]:
df.head()

In [ ]:
df['Primary Fur Color'] = df['Primary Fur Color'].fillna('Unknown')

In [ ]:
le_color = LabelEncoder()
df['Primary Fur Color'] = le_color.fit_transform(df['Primary Fur Color'])

In [ ]:
features = ['Longitude', 'Latitude', 'Hectare Squirrel Number']
X = df[features]
y = df['Primary Fur Color']

In [ ]:
mask_missing = df['Primary Fur Color'] == le_color.transform(['Unknown'])[0]
X_missing = X[mask_missing]
y_missing = y[mask_missing]
X_known = X[~mask_missing]
y_known = y[~mask_missing]

In [ ]:
model = Pipeline(steps=[
    ('scaler', StandardScaler()),  # Standardize features
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [ ]:
cross_val_scores = cross_val_score(model, X_known, y_known, cv=5)
print(f'Cross-validation scores: {cross_val_scores}')
print(f'Average cross-validation score: {cross_val_scores.mean()}')

In [ ]:
model.fit(X_known, y_known)

In [ ]:
y_missing_pred = model.predict(X_missing)

In [ ]:
df.loc[mask_missing, 'Primary Fur Color'] = y_missing_pred

In [ ]:
df['Primary Fur Color'] = le_color.inverse_transform(df['Primary Fur Color'])

In [ ]:
df.to_csv('updated_dataset.csv', index=False)

In [ ]:
df.head()

In [ ]:
df["Primary Fur Color"].unique()

In [ ]:
updated_df = pd.read_csv("updated_dataset.csv")

In [ ]:
central_park_lat = 40.785091
central_park_lon = -73.968285
m = folium.Map(location=[central_park_lat, central_park_lon], zoom_start=13)

In [ ]:
color_map = {
    'Gray': '295F98',
    'Cinnamon': 'brown',
    'Black': 'black',
    'Unknown': 'lightblue'
}

In [ ]:
for _, row in df.iterrows():
    fur_color = row['Primary Fur Color']
    color = color_map.get(fur_color, 'lightblue')  # Default to lightblue if color not found
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f'Fur Color: {fur_color}'
    ).add_to(m)

In [ ]:
m.save('squirrel_map.html')

In [ ]:
m